# 📔 Jupyter Notebook: RAG de Alto Nivel con LangChain y FAISS

## Objetivo de la Clase

Tras haber construido un sistema RAG desde cero, ahora vamos a resolver el mismo problema utilizando herramientas estándar de la industria: **LangChain** y **FAISS**.

El objetivo es comprender cómo los frameworks de alto nivel abstraen la complejidad y nos permiten construir sistemas RAG robustos y eficientes con mucho menos código. Compararemos directamente los componentes de LangChain con las funciones que creamos manualmente.

### De lo Manual a LangChain: Un Mapeo

| Tarea | Nuestra Implementación Manual | Abstracción en LangChain |
| :--- | :--- | :--- |
| **Cargar Datos (PDF)** | `extraer_texto_de_pdf()` | `Document Loaders` (ej. `PyMuPDFLoader`) |
| **Fragmentar Texto** | `dividir_en_chunks()` | `Text Splitters` (ej. `RecursiveCharacterTextSplitter`) |
| **Generar Embeddings** | `get_gemini_embedding()`, etc. | `Embeddings Wrappers` (ej. `GoogleGenerativeAIEmbeddings`) |
| **Almacenar y Buscar** | `SQLite` + `cosine_similarity` | `Vector Stores` (ej. `FAISS`) |
| **Formatear Prompt** | `build_prompt()` | `Prompt Templates` (ej. `ChatPromptTemplate`) |
| **Llamar al LLM** | `get_llm_response()` | `LLM/ChatModel Wrappers` (ej. `ChatGoogleGenerativeAI`) |
| **Orquestar el Flujo**| Script lineal | `Chains` (LCEL - LangChain Expression Language) |

### **Paso 0: Instalación de Dependencias**

Las dependencias de LangChain son modulares. Necesitamos instalar el paquete principal y los específicos para los componentes que usaremos (OpenAI, Gemini, Cohere, FAISS, etc.).

In [ ]:
# Descomenta y ejecuta la siguiente línea si no tienes instaladas las librerías
# !pip install langchain langchain-openai langchain-google-genai langchain-cohere langchain-community faiss-cpu pypdf fitz

### **Paso 1: Configuración de APIs y Creación del PDF**

Este paso inicial es similar. Configuramos nuestras claves de API y creamos el mismo documento PDF de ejemplo para tener una fuente de datos consistente.

In [ ]:
import os
import fitz  # PyMuPDF

# --- CONFIGURACIÓN DE LAS API KEYS ---
os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"
os.environ["GEMINI_API_KEY"] = "YOUR_GEMINI_API_KEY"
os.environ["COHERE_API_KEY"] = "YOUR_COHERE_API_KEY"

print("API keys configuradas.")

# --- CREACIÓN DEL PDF DE EJEMPLO ---
# Reutilizamos la función y el texto del notebook anterior
def crear_pdf_de_ejemplo(nombre_archivo, texto):
    doc = fitz.open()
    pagina = doc.new_page()
    rect = fitz.Rect(50, 50, 550, 800)
    # PyMuPDF > 1.24.2 usa insert_textbox en lugar de insert_text
    try:
        pagina.insert_textbox(texto, rect, fontsize=12, fontname="helv")
    except AttributeError:
        pagina.insert_text(texto, rect, fontsize=12, fontname="helv")
    doc.save(nombre_archivo)
    doc.close()

texto_ia = """
Historia de la Inteligencia Artificial

La historia de la inteligencia artificial (IA) es fascinante y se remonta a la antigüedad... [El mismo texto largo del notebook anterior]
"""
PDF_FILENAME = "historia_ia_langchain.pdf"
crear_pdf_de_ejemplo(PDF_FILENAME, texto_ia)
print(f"PDF '{PDF_FILENAME}' creado para el ejemplo con LangChain.")

### **Paso 2: Carga y División de Datos con LangChain**

Aquí vemos la primera gran simplificación. Usamos un `DocumentLoader` para cargar el PDF y un `TextSplitter` para dividirlo en chunks. LangChain maneja la iteración sobre las páginas y la lógica de la división.

In [ ]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 1. Cargar el documento
loader = PyMuPDFLoader(PDF_FILENAME)
documents = loader.load()

# 2. Dividir el documento en chunks
# RecursiveCharacterTextSplitter es más robusto que un split simple.
# Intenta dividir por párrafos, luego por líneas, etc.
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=100  # Un pequeño solapamiento ayuda a mantener el contexto entre chunks
)
chunks = text_splitter.split_documents(documents)

print(f"Documento cargado y dividido en {len(chunks)} chunks.")
print("\n--- Metadatos del Primer Chunk ---")
print(chunks[0].metadata)
print("\n--- Contenido del Primer Chunk ---")
print(chunks[0].page_content[:300] + "...")

### **Paso 3: Embeddings y Creación del Vector Store (FAISS)**

Este es el paso más impactante. Con una sola línea de código, generamos los embeddings para todos los chunks y los indexamos en un **vector store FAISS** en memoria.

**FAISS (Facebook AI Similarity Search)** es una librería altamente optimizada para la búsqueda eficiente de similitud entre vectores. LangChain se integra perfectamente con ella.

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import FAISS

# Seleccionamos el modelo de embeddings (podríamos usar OpenAIEmbeddings o CohereEmbeddings también)
embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")

# Creamos el Vector Store FAISS a partir de los chunks y el modelo de embeddings
# ¡Esta línea reemplaza toda nuestra lógica de base de datos y serialización!
print("Creando el vector store FAISS...")
vectorstore = FAISS.from_documents(chunks, embeddings)
print("Vector store creado con éxito.")

# FAISS puede guardarse en disco y cargarse después
# vectorstore.save_local("faiss_index")
# new_vectorstore = FAISS.load_local("faiss_index", embeddings)

### **Paso 4: Creación de la Cadena RAG con LCEL**

**LCEL (LangChain Expression Language)** es la forma moderna y declarativa de construir cadenas en LangChain. Nos permite "unir" componentes con el símbolo `|` (pipe).

Construiremos una cadena que:
1.  Toma una pregunta.
2.  Usa un **retriever** (creado desde nuestro vector store) para obtener los documentos relevantes.
3.  Formatea un prompt con la pregunta y los documentos recuperados.
4.  Pasa el prompt al LLM.
5.  Parsea la salida para obtener una respuesta limpia.

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain import hub

# 1. Definimos el LLM que generará la respuesta
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

# 2. Creamos un "retriever" desde nuestro vector store
# Esto reemplaza nuestra función `retrieve_top_k`
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

# 3. Usamos un prompt de la comunidad de LangChain (o creamos uno propio)
prompt = hub.pull("rlm/rag-prompt")

# 4. Creamos la cadena usando LCEL
def format_docs(docs):
    # Formatea los documentos recuperados en un solo string
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
print("Cadena RAG creada.")

### **Paso 5: Ejecución de la Cadena y Comparación de Modelos**

Ahora, simplemente "invocamos" nuestra cadena con una pregunta. LangChain se encarga de todo el proceso de RAG de forma transparente.

In [ ]:
user_query = "¿Quiénes son considerados los 'padrinos de la IA' y por qué?"

print(f"Ejecutando la cadena con la query: '{user_query}'")
print("--- RESPUESTA (Google Gemini) ---")

# Invocamos la cadena y obtenemos la respuesta
response = rag_chain.invoke(user_query)
print(response)

# --- Probando con OpenAI (ejemplo de modularidad) ---
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

print("\n--- Reconfigurando la cadena para usar OpenAI ---")

# 1. Cambiamos el modelo de embeddings y creamos un nuevo vector store
embeddings_openai = OpenAIEmbeddings(model="text-embedding-3-small")
vectorstore_openai = FAISS.from_documents(chunks, embeddings_openai)

# 2. Cambiamos el LLM
llm_openai = ChatOpenAI(model="gpt-4o-mini")

# 3. Creamos el nuevo retriever y la nueva cadena
retriever_openai = vectorstore_openai.as_retriever()
rag_chain_openai = (
    {"context": retriever_openai | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm_openai
    | StrOutputParser()
)

print("--- RESPUESTA (OpenAI GPT-4o-mini) ---")
response_openai = rag_chain_openai.invoke(user_query)
print(response_openai)

## Conclusiones Finales: ¿Por qué usar un Framework?

Como hemos visto, LangChain y FAISS reducen drásticamente la cantidad de código y la complejidad para construir un sistema RAG.

- **Velocidad de Desarrollo:** Pasamos de implementar manualmente la lógica de la base de datos, el chunking y la recuperación a usar componentes pre-construidos y optimizados.
- **Robustez y Mantenimiento:** El código es más limpio, más declarativo y más fácil de mantener. Aprovechamos las mejoras y correcciones de errores de la comunidad de LangChain.
- **Rendimiento:** FAISS proporciona una búsqueda de similitud mucho más rápida que nuestro cálculo manual de similitud del coseno sobre todos los elementos, lo cual es crucial para grandes volúmenes de datos.
- **Modularidad:** Cambiar un componente (como el LLM o el modelo de embeddings) es trivial, como demostramos al pasar de Gemini a OpenAI.

Sin embargo, la lección más importante es que **haber construido el sistema manualmente primero nos da una comprensión profunda de lo que estas herramientas hacen por debajo**. Este conocimiento es invaluable para depurar problemas, optimizar el rendimiento y tomar decisiones informadas sobre qué componentes usar en un proyecto real.